In [1]:
import os
import os.path
import numpy as np
import pandas as pd
from yahoo_fin import stock_info as si
from VBTT2_IO.IO import read_list,delete_then_get_model_from_bucket,delete_blob,read_config_file,instantiate_logging,save_dataframe_to_bucket,read_dataframe_from_bucket,upload_file_to_bucket
from VBTT2_Predict.Predict import predict_ticker,YF_datetime
from VBTT2_SP500.SP500 import read_create_write_SP500, get_SP500,get_all_tickers_sector,get_all_tickers_industry
from VBTT2_Features.Features import get_yf_dataframe,initialize_data,create_train_test_set,create_predict_set
from VBTT2_Model.Model import  model_predict, add_buy_sell_to_prediction,balanced_accuracy

In [2]:
!export GOOGLE_APPLICATION_CREDENTIALS='/home/steve/Downloads/creds.json'

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/steve/Downloads/creds.json"

In [4]:
print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

/home/steve/Downloads/creds.json


In [5]:
pip install yahoo_fin --upgrade

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
Results=read_dataframe_from_bucket("Results.csv")


In [7]:
Results

,Ticker,Date,Observed,Predicted for,Predicted,Recommended,Return %,Sector,Model
0,TTWO,2022-11-30,105.690002,2022-11-30,100.055661,Sell,-4.106352,Communication Services,Lasso()
1,TTWO,2022-12-01,106.400002,2022-12-01,105.485446,Buy,0.671775,Communication Services,Lasso()
2,TTWO,2022-12-02,108.760002,2022-12-02,106.993596,Buy,2.218046,Communication Services,Lasso()
3,TTWO,2022-12-05,106.980003,2022-12-05,108.497596,Buy,-1.636630,Communication Services,Lasso()
4,TTWO,2022-12-06,101.269997,2022-12-06,106.790140,Sell,5.638399,Communication Services,Lasso()
...,...,...,...,...,...,...,...,...,...
15,TTWO,2022-12-21,99.879997,2022-12-21,99.424772,Sell,-1.772123,Communication Services,Lasso()
16,TTWO,2022-12-22,99.269997,2022-12-22,99.797860,Buy,-0.610734,Communication Services,Lasso()
17,TTWO,2022-12-23,99.830002,2022-12-23,99.798034,Buy,0.564123,Communication Services,Lasso()
18,TTWO,2022-12-27,99.379997,2022-12-27,100.668609,Buy,-0.450771,Communication Services,Lasso()


In [8]:
print(Results.dtypes)

Ticker            object
Date              object
Observed         float64
Predicted for     object
Predicted        float64
Recommended       object
Return %         float64
Sector            object
Model             object
dtype: object


In [9]:
pip install google.api.core

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install google-cloud-storage

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### New function to generate Json file - the jsonfilename could be a parameter on the config file later

In [11]:
filename_json=read_config_file()[5]
get_SP500(filename_json)

'JSON SP500.json was generated  successfully.'

In [104]:
#ancienne methode avec si.get_company_info qui ne marche plus
df = si.get_company_info("GOOG")  # a utiliser pour trouver le secteur
df = df.reset_index()
sector = df.loc[df['Breakdown'].isin(['sector'])]  # from bamboolib to extract sector
sector = sector.iloc[0, 1]  # this is to extract just the value
industry = df.loc[df['Breakdown'].isin(['industry'])]
industry = industry.iloc[0, 1]


from yahoo_fin import stock_info as si
si.get_company_info('aapl')

TypeError: string indices must be integers

In [116]:
#nouvelle methode avec yfinance remplacant la methode ci-dessus

import yfinance as yf

ticker = yf.Ticker("gen")
company_info = ticker.info
print(company_info['sector'])
print(company_info['industry'])

KeyError: 'sector'

In [118]:
import yfinance as yf
import time
#tickers_list=si.tickers_sp500()
tickers_list=['AAPL','GOOG','GEN','NFLX','AM','AT','AMZ','K','CAG','CTRA']
SP500_list = []
for ticker in tickers_list:

    ticker_company_info = yf.Ticker(ticker).info
    try:
        sector=ticker_company_info['sector']
    except:
        print(f"no sector for {ticker}")
    else:
        sector=ticker_company_info['sector']
        industry=ticker_company_info['industry']
        print([ticker,sector,industry])
        SP500_list.append([ticker,sector,industry])

SP500_list = np.array(SP500_list)


['AAPL', 'Technology', 'Consumer Electronics']
['GOOG', 'Communication Services', 'Internet Content & Information']
['GEN', 'Technology', 'Software—Infrastructure']
['NFLX', 'Communication Services', 'Entertainment']
['AM', 'Energy', 'Oil & Gas Midstream']
- AT: No summary info found, symbol may be delisted
no sector for AT
- AMZ: No summary info found, symbol may be delisted
no sector for AMZ
['K', 'Consumer Defensive', 'Packaged Foods']
['CAG', 'Consumer Defensive', 'Packaged Foods']
['CTRA', 'Energy', 'Oil & Gas E&P']


In [120]:
print(SP500_list)

[['AAPL' 'Technology' 'Consumer Electronics']
 ['GOOG' 'Communication Services' 'Internet Content & Information']
 ['GEN' 'Technology' 'Software—Infrastructure']
 ['NFLX' 'Communication Services' 'Entertainment']
 ['AM' 'Energy' 'Oil & Gas Midstream']
 ['K' 'Consumer Defensive' 'Packaged Foods']
 ['CAG' 'Consumer Defensive' 'Packaged Foods']
 ['CTRA' 'Energy' 'Oil & Gas E&P']]


In [ ]:
SP500_list

In [92]:
pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 KB 7.6 MB/s eta 0:00:00
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 KB 9.0 MB/s eta 0:00:00a 0:00:01
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
'''
df = si.get_company_info("AAPL")  # a utiliser pour trouver le secteur
df = df.reset_index()
sector = df.loc[df['Breakdown'].isin(['sector'])]  # from bamboolib to extract sector
sector = sector.iloc[0, 1]  # this is to extract just the value
industry = df.loc[df['Breakdown'].isin(['industry'])]
industry = industry.iloc[0, 1]
'''

'\ndf = si.get_company_info("AAPL")  # a utiliser pour trouver le secteur\ndf = df.reset_index()\nsector = df.loc[df[\'Breakdown\'].isin([\'sector\'])]  # from bamboolib to extract sector\nsector = sector.iloc[0, 1]  # this is to extract just the value\nindustry = df.loc[df[\'Breakdown\'].isin([\'industry\'])]\nindustry = industry.iloc[0, 1]\n'

In [94]:
company_info

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 164000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. In addition, the company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. Further, it provides AppleCare support and cloud services store services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. Additionally, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experience with on-demand rad

In [15]:
read_config_file()

('0.1',
 ['^tnx', '^GSPC', 'CL=F'],
 '*',
 'Lasso(alpha=1.0)',
 'stock-363101',
 'SP500.json')

# extracting sectors and industries from Jsonfile

In [16]:
filename_json=read_config_file()[5]
SP500_list=read_list(filename_json)
SP500_list = np.array(SP500_list,dtype=object)
print(SP500_list)

[['A' 'Healthcare' 'Diagnostics & Research']
 ['AAL' 'Industrials' 'Airlines']
 ['AAP' 'Consumer Cyclical' 'Specialty Retail']
 ...
 ['ZBRA' 'Technology' 'Communication Equipment']
 ['ZION' 'Financial Services' 'Banks—Regional']
 ['ZTS' 'Healthcare' 'Drug Manufacturers—Specialty & Generic']]


In [17]:
SP500_list=read_list(filename_json)
SP500_list = np.array(SP500_list,dtype=object)
print(SP500_list)

[['A' 'Healthcare' 'Diagnostics & Research']
 ['AAL' 'Industrials' 'Airlines']
 ['AAP' 'Consumer Cyclical' 'Specialty Retail']
 ...
 ['ZBRA' 'Technology' 'Communication Equipment']
 ['ZION' 'Financial Services' 'Banks—Regional']
 ['ZTS' 'Healthcare' 'Drug Manufacturers—Specialty & Generic']]


In [18]:
SP500_tickers=(SP500_list[:,0])
SP500_sectors=(SP500_list[:,1])
SP500_industries=(SP500_list[:,2])

In [19]:
print(SP500_tickers)
print(SP500_sectors)
print(SP500_industries)

['A' 'AAL' 'AAP' 'AAPL' 'ABBV' 'ABC' 'ABMD' 'ABT' 'ACN' 'ADBE' 'ADI' 'ADM'
 'ADP' 'ADSK' 'AEE' 'AEP' 'AES' 'AFL' 'AIG' 'AIZ' 'AJG' 'AKAM' 'ALB'
 'ALGN' 'ALK' 'ALL' 'ALLE' 'AMAT' 'AMCR' 'AMD' 'AME' 'AMGN' 'AMP' 'AMT'
 'AMZN' 'ANET' 'ANSS' 'AON' 'AOS' 'APA' 'APD' 'APH' 'APTV' 'ARE' 'ATO'
 'ATVI' 'AVB' 'AVGO' 'AVY' 'AWK' 'AXP' 'AZO' 'BA' 'BAC' 'BALL' 'BAX'
 'BBWI' 'BBY' 'BDX' 'BEN' 'BF-B' 'BIIB' 'BIO' 'BK' 'BKNG' 'BKR' 'BLK'
 'BMY' 'BR' 'BRK-B' 'BRO' 'BSX' 'BWA' 'BXP' 'C' 'CAG' 'CAH' 'CARR' 'CAT'
 'CB' 'CBOE' 'CBRE' 'CCI' 'CCL' 'CDAY' 'CDNS' 'CDW' 'CE' 'CEG' 'CF' 'CFG'
 'CHD' 'CHRW' 'CHTR' 'CI' 'CINF' 'CL' 'CLX' 'CMA' 'CMCSA' 'CME' 'CMG'
 'CMI' 'CMS' 'CNC' 'CNP' 'COF' 'COO' 'COP' 'COST' 'CPB' 'CPRT' 'CPT' 'CRL'
 'CRM' 'CSCO' 'CSGP' 'CSX' 'CTAS' 'CTLT' 'CTRA' 'CTSH' 'CTVA' 'CVS' 'CVX'
 'CZR' 'D' 'DAL' 'DD' 'DE' 'DFS' 'DG' 'DGX' 'DHI' 'DHR' 'DIS' 'DISH' 'DLR'
 'DLTR' 'DOV' 'DOW' 'DPZ' 'DRI' 'DTE' 'DUK' 'DVA' 'DVN' 'DXC' 'DXCM' 'EA'
 'EBAY' 'ECL' 'ED' 'EFX' 'EIX' 'EL' 'ELV' 'EMN' 'EMR' 'ENPH

In [20]:
print(len(SP500_tickers))
print(len(SP500_sectors))
print(len(SP500_industries))

503
503
503


In [21]:
a=get_all_tickers_sector('Basic Materials').tolist()


In [22]:
print(a)

['ALB', 'APD', 'CE', 'CF', 'CTVA', 'DD', 'DOW', 'ECL', 'EMN', 'FCX', 'FMC', 'IFF', 'LIN', 'LYB', 'MLM', 'MOS', 'NEM', 'NUE', 'PPG', 'SHW', 'VMC']


In [23]:
test=si.get_data("aapl")

In [24]:
'''
si.get_company_officers("aapl")
'''

'\nsi.get_company_officers("aapl")\n'

In [25]:
get_all_tickers_industry('Farm Products')

array([['ADM'],
       ['TSN']], dtype='<U38')

In [5]:
years=5
days=years*360
nb_predict_days=30
lags=30



In [27]:
filename_json=read_config_file()[5]
yesterday, start_date, train_date_start, train_date_last, test_date_start, test_date_last,days=initialize_data(days,lags,nb_predict_days)



In [28]:
SP500_list=read_list(filename_json)
SP500_list = np.array(SP500_list,dtype=object)
unique_SP500_sectors=np.unique(SP500_list[:,1])
unique_SP500_tickers=np.unique(SP500_list[:,0])

In [121]:
def generate_results(ticker,sector,Predictions):
    DF_Recommendations = []
    #blc_accuracy=[0]  # render for view html need a first element to be 0
    #blc_accuracy.append(round(balanced_accuracy(ticker, Predictions.iloc[:-1]),3))
    DF_Recommendations.append([ticker, "", "", "", balanced_accuracy(ticker, Predictions.iloc[:-1]),""])
    Recommendations = pd.DataFrame(DF_Recommendations, columns=["Ticker", 'Predicted for', 'Predicted', "Recommended", "Accuracy","Return%"])



    #avg_return = [0]  # render for view html need a first element to be 0
    # results = Predictions[['y_test', 'y_pred', 'ticker', 'y_predb','y_recommend']][Predictions['ticker'] == ticker]
    results = Predictions[['y_test', 'y_pred', 'ticker', 'y_predb', 'y_recommend', 'daily return %']][Predictions['ticker'] == ticker]
    # results['y_test'].iloc[-1]=np.nan  # pas la bonne methode
    results.loc[results.index[-1], 'y_test'] = np.nan
    results['daily return %'].iloc[-1] = np.nan  # pas la bonne methode
    results.loc[results.index[-1], 'daily return %'] = np.nan
    date_predict = YF_datetime() #can be confusing but yesterday is datetime.now. good thing is to take latest in results +1
    date_predict = date_predict.strftime("%Y/%m/%d")
    ticker_predicted = results.iloc[-1]['y_pred']  # this is the last row containing result
    ticker_recommend = results.iloc[-1]['y_recommend']  # this is the last row containing result
    Recommendations.loc[Recommendations['Ticker'] == ticker, 'Predicted for'] = date_predict  # to change content of a cell
    Recommendations.loc[Recommendations['Ticker'] == ticker, 'Predicted'] = ticker_predicted
    Recommendations.loc[Recommendations['Ticker'] == ticker, 'Recommended'] = ticker_recommend
    Predictions.loc[(Predictions['Date'] == date_predict) & (Predictions['ticker'] == ticker), 'y_test'] = np.nan
    Predictions.loc[(Predictions['Date'] == date_predict) & (Predictions['ticker'] == ticker), 'daily return %'] = np.nan

    ticker_return = round(Predictions['daily return %'][Predictions['ticker'] == ticker].iloc[:-1].mean(), 2)
    Recommendations.loc[Recommendations['Ticker'] == ticker, 'Return%'] = ticker_return

    #avg_return.append(ticker_return) # we are creating a list of the return
    # print(f"Prediction for {yesterday+timedelta(1)} -- ticker: {ticker} {'**resultat**'}\n {resultat.tail()}\n\n")


    Results = Predictions[['ticker', 'Date', 'y_test', 'Predicted for', 'y_pred', 'y_recommend', 'daily return %']]
    Results = Results.rename(
        columns={'ticker': 'Ticker', 'y_test': 'Observed', 'y_pred': 'Predicted', 'y_recommend': 'Recommended',
                 'daily return %': 'Return %'})

    Results['Sector']=sector
    Results['Model']=MODEL

    #below is not uses for this module but might be useful
    #date_range = [0]
    #date_range.append(Results.iloc[0]['Date'].strftime("%Y/%m/%d"))
    #date_range.append(Results.iloc[-2]['Date'].strftime("%Y/%m/%d"))

    return Results


In [122]:
# import the regressors
version, additional_data, regressor, MODEL,bucket,filename_json=read_config_file()


if MODEL == 'DecisionTree()':
    from sklearn.tree import DecisionTreeRegressor
    MODEL = DecisionTreeRegressor()
elif MODEL == 'LinearRegression()':
    from sklearn.linear_model import LinearRegression
    MODEL = LinearRegression()
elif MODEL == 'svm.SVR()':
    from sklearn import svm
    MODEL = svm.SVR()
elif MODEL == "DecisionTree(max_depth=5)":
    from sklearn.tree import DecisionTreeRegressor
    MODEL = DecisionTreeRegressor(max_depth=5)
elif MODEL == "Ridge(alpha=1.0)":
    from sklearn.linear_model import Ridge
    MODEL = Ridge(alpha=1.0)
elif MODEL == "Lasso(alpha=1.0)":
    from sklearn.linear_model import Lasso
    MODEL = Lasso(alpha=1.0)
elif MODEL == "xgboost":
    import xgboost as xgb
    MODEL = xgb.XGBClassifier()
elif MODEL == "RandomForestRegressor(n_estimators=100)":
    from sklearn.ensemble import RandomForestRegressor
    MODEL = RandomForestRegressor(n_estimators=100)
else:
    # MODEL=='LinearRegression'
    from sklearn.linear_model import LinearRegression
    MODEL = LinearRegression()






In [ ]:

logger=instantiate_logging()
predictions = pd.DataFrame()  # to store predictionsdfpredictions = pd.DataFrame()  # to store predictions
Results=pd.DataFrame()
for sector in unique_SP500_sectors:

    ticker_list_for_models=get_all_tickers_sector(sector).tolist()


    additional_data = read_config_file()[1]
    tickers_in_sector_extended = np.concatenate((ticker_list_for_models, additional_data), axis=None)
    tickers_in_sector_extended = tickers_in_sector_extended.tolist()

        ### validation if we have everything
    logger.log_text(f"Function Model_train_Save|  Input ->Sector {sector}")
    logger.log_text(f"Function Model_train_Save|  Input ->ticker list {ticker_list_for_models}")
    logger.log_text(f"Function Model_train_Save|  Input ->ticker list extended {tickers_in_sector_extended")
    logger.log_text(f"Function Model_train_Save|  Input ->years {years}")
    logger.log_text(f"Function Model_train_Save|  Input->lags {lags}")
    logger.log_text(f"Function Model_train_Save|  Input ->predict days {nb_predict_days}")
    logger.log_text(f"Function Model_train_Save|  Period->yesterday {yesterday}")
    logger.log_text(f"Function Model_train_Save|  Period->train_date_start {train_date_start}")
    logger.log_text(f"Function Model_train_Save|  Period->train_date_last {train_date_last}")
    logger.log_text(f"Function Model_train_Save|  Period->test_date_start {test_date_start}")
    logger.log_text(f"Function Model_train_Save|  Period->test_date_last {test_date_last}")

    logger.log_text(f"Function Model_train_Save| Tickers for our model {ticker_list_for_models}")
    logger.log_text(f"Function Model_train_Save| Additional data  we add to the tickers for the model {additional_data}")

    Logger.log_text(f"Function Model_train_Save| Model VALIDATE - This is the number of training days of the train dataset {days}")







    matrix_features_sector = get_yf_dataframe(tickers_in_sector_extended, days)
    matrix_features_sector = matrix_features_sector.reindex(sorted(matrix_features_sector.columns), axis=1)


    for ticker in ticker_list_for_models:



        X_train, y_train, X_test, y_test, df_filtered = create_train_test_set(ticker, matrix_features_sector, lags, additional_data, days, nb_predict_days)
        print(MODEL)
        print(ticker)
        MODEL.fit(X_train, y_train)
        # save the model to disk
        #filename = ticker + '_modelnew.sav'
        #joblib.dump(MODEL, filename)
        #upload_file_to_bucket(filename)
        
        X_test, y_test, df_filtered = create_predict_set(ticker, matrix_features_sector, lags, nb_predict_days,additional_data)
        
        temp_pred = model_predict(MODEL, ticker, X_test, y_test)


        # ===
        temp_pred.drop(labels=[0], inplace=True)


        from datetime import timedelta

        df_filtered2 = pd.DataFrame(df_filtered[ticker])

        df_filtered2 = df_filtered2.reset_index()

        # Deleted 1 row in df_filtered2
        df_filtered2.drop(labels=[0], inplace=True)

        # Renamed columns Date
        df_filtered2.rename(columns={'index': 'Date'}, inplace=True)

        df_filtered2['Predicted for'] = df_filtered2['Date']
        # df_filtered2['Prediction for']=df_filtered2['Date']
        df_filtered2 = df_filtered2[['Date', 'Predicted for']]

        # Step: Copy a dataframe column

        temp_pred2 = pd.concat([temp_pred, df_filtered2], axis=1)

        # ===

        #adding other information
        temp_pred2['Sector']=sector
        temp_pred2["Model"]=MODEL
        # ===
        predictions = predictions.append(temp_pred2, ignore_index=True)  # this is to store in the master pandas list
        
        
        # add binary buy=1 and sell=0
        Predictions = add_buy_sell_to_prediction(predictions,[ticker])
        # lag_lagged is a DF containing predictions + buy and Sell label
        #print("Predictions -->",Predictions)


        '''
        #Etape finale - 
        print(ticker)
        print(sector)
        print(ticker_list_for_models)
        #print(matrix_features_sector)
        print(lags)
        print(additional_data)
        print(days)
        print(nb_predict_days)
        '''
        
        Results_for_ticker=generate_results(ticker,sector,Predictions)
        Results = Results.append(Results_for_ticker.iloc[-1], ignore_index=True)  # this is to store in the master pandas list. Not [-1] because we want to add only the latest prediction
        





In [86]:
#predictions

In [33]:
#Predictions

In [87]:
Results

,Ticker,Date,Observed,Predicted for,Predicted,Recommended,Return %,Sector,Model
0,ALB,2022-12-29,NaN,2022-12-29,214.219937,Sell,NaN,Basic Materials,Lasso()
1,APD,2022-12-29,NaN,2022-12-29,309.113510,Sell,NaN,Basic Materials,Lasso()
2,CE,2022-12-29,NaN,2022-12-29,100.149951,Sell,NaN,Basic Materials,Lasso()
3,CF,2022-12-29,NaN,2022-12-29,86.343746,Sell,NaN,Basic Materials,Lasso()
4,CTVA,2022-12-29,NaN,2022-12-29,58.323030,Sell,NaN,Basic Materials,Lasso()
...,...,...,...,...,...,...,...,...,...
498,PPL,2022-12-29,NaN,2022-12-29,29.180259,Sell,NaN,Utilities,Lasso()
499,SO,2022-12-29,NaN,2022-12-29,71.178197,Sell,NaN,Utilities,Lasso()
500,SRE,2022-12-29,NaN,2022-12-29,156.069442,Sell,NaN,Utilities,Lasso()
501,WEC,2022-12-29,NaN,2022-12-29,94.727418,Sell,NaN,Utilities,Lasso()


In [35]:
#Results_for_ticker=generate_results(ticker,sector)

# Tests section

In [123]:
years=5
days=years*360
nb_predict_days=30
lags=30

ticker='TTWO'
sector="Communication Services"
yesterday, start_date, train_date_start, train_date_last, test_date_start, test_date_last,days=initialize_data(days,lags,nb_predict_days)

unique_SP500_tickers=[ticker]
predictions = pd.DataFrame()
print(ticker)
print(sector)
#print(matrix_features_sector)
print(lags)
print(additional_data)
print(days)
print(nb_predict_days)


print(MODEL)
print(sector)
print(ticker)

ticker_list_for_models=get_all_tickers_sector(sector).tolist()
additional_data = read_config_file()[1]
tickers_in_sector_extended = np.concatenate((ticker_list_for_models, additional_data), axis=None)
tickers_in_sector_extended = tickers_in_sector_extended.tolist()
matrix_features_sector = get_yf_dataframe(tickers_in_sector_extended, days)
matrix_features_sector = matrix_features_sector.reindex(sorted(matrix_features_sector.columns), axis=1)



X_train, y_train, X_test, y_test, df_filtered = create_train_test_set(ticker, matrix_features_sector, lags, additional_data, days, nb_predict_days)

print(lags*2)
print(nb_predict_days)
print(test_date_start)
print (test_date_last)

print("Model_train", df_filtered)

MODEL.fit(X_train, y_train)
X_test, y_test, df_filtered = create_predict_set(ticker, matrix_features_sector, lags, nb_predict_days,additional_data)
temp_pred = model_predict(MODEL, ticker, X_test, y_test)
print("===============================")
print("Model_predict", df_filtered)

# ===
temp_pred.drop(labels=[0], inplace=True)

from datetime import timedelta
df_filtered2 = pd.DataFrame(df_filtered[ticker])

df_filtered2 = df_filtered2.reset_index()

# Deleted 1 row in df_filtered2
df_filtered2.drop(labels=[0], inplace=True)

#Renamed columns Date
df_filtered2.rename(columns={'index': 'Date'}, inplace=True)


df_filtered2['Predicted for'] = df_filtered2['Date']
# df_filtered2['Prediction for']=df_filtered2['Date']
df_filtered2 = df_filtered2[['Date', 'Predicted for']]

# Step: Copy a dataframe column

temp_pred2 = pd.concat([temp_pred, df_filtered2], axis=1)

# ===
#adding other information
temp_pred2['Sector']=sector
temp_pred2["Model"]=MODEL



predictions = predictions.append(temp_pred2, ignore_index=True)  # this is to store in the master pandas list


# add binary buy=1 and sell=0
Predictions = add_buy_sell_to_prediction(predictions,[ticker])
# lag_lagged is a DF containing predictions + buy and Sell label


#Etape finale - 

Results=generate_results(ticker,sector,Predictions)

TTWO
Communication Services
30
['^tnx', '^GSPC', 'CL=F']
1800
30
Lasso()
Communication Services
TTWO
60
30
2022-11-29
2022-12-29
Model_train             CL=F_lag001  CL=F_lag002  CL=F_lag003  CL=F_lag004  CL=F_lag005  \
2022-11-29    77.239998    76.279999    77.940002    80.949997    79.730003   
2022-11-30    78.199997    77.239998    76.279999    77.940002    80.949997   
2022-12-01    80.550003    78.199997    77.239998    76.279999    77.940002   
2022-12-02    81.220001    80.550003    78.199997    77.239998    76.279999   
2022-12-05    79.980003    81.220001    80.550003    78.199997    77.239998   
2022-12-06    76.930000    79.980003    81.220001    80.550003    78.199997   
2022-12-07    74.250000    76.930000    79.980003    81.220001    80.550003   
2022-12-08    72.010002    74.250000    76.930000    79.980003    81.220001   
2022-12-09    71.459999    72.010002    74.250000    76.930000    79.980003   
2022-12-12    71.019997    71.459999    72.010002    74.250000    76.

/home/steve/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.585e+03, tolerance: 1.025e+02
  model = cd_fast.enet_coordinate_descent(
/home/steve/Documents/McGill/VBTT3/VBTT2_Features/Features.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_last)  # we append last date to end
/tmp/ipykernel_266469/4259233876.py:79: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(temp_pred2, ignore_index=True)  # this is to store in the master pandas list
/home/steve/Documents/McGill/VBTT3/VBTT2_Model/Model.py:187: FutureWarning: The frame.append method is

In [37]:
df_filtered

,CL=F_lag001,CL=F_lag002,CL=F_lag003,CL=F_lag004,CL=F_lag005,CL=F_lag006,CL=F_lag007,CL=F_lag008,CL=F_lag009,CL=F_lag010,...,^tnx_lag021,^tnx_lag022,^tnx_lag023,^tnx_lag024,^tnx_lag025,^tnx_lag026,^tnx_lag027,^tnx_lag028,^tnx_lag029,^tnx_lag030
2022-11-28,76.279999,77.940002,80.949997,79.730003,80.080002,81.639999,85.589996,86.919998,85.870003,88.959999,...,3.937,4.015,4.108,4.234,4.213,4.226,4.127,3.998,4.015,4.010
2022-11-29,77.239998,76.279999,77.940002,80.949997,79.730003,80.080002,81.639999,85.589996,86.919998,85.870003,...,4.010,3.937,4.015,4.108,4.234,4.213,4.226,4.127,3.998,4.015
2022-11-30,78.199997,77.239998,76.279999,77.940002,80.949997,79.730003,80.080002,81.639999,85.589996,86.919998,...,4.077,4.010,3.937,4.015,4.108,4.234,4.213,4.226,4.127,3.998
2022-12-01,80.550003,78.199997,77.239998,76.279999,77.940002,80.949997,79.730003,80.080002,81.639999,85.589996,...,4.052,4.077,4.010,3.937,4.015,4.108,4.234,4.213,4.226,4.127
2022-12-02,81.220001,80.550003,78.199997,77.239998,76.279999,77.940002,80.949997,79.730003,80.080002,81.639999,...,4.059,4.052,4.077,4.010,3.937,4.015,4.108,4.234,4.213,4.226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-21,76.089996,75.190002,74.290001,76.110001,77.279999,75.389999,73.169998,71.019997,71.459999,72.010002,...,3.825,3.818,3.775,3.692,3.799,3.865,3.813,3.829,4.151,4.126
2022-12-22,78.290001,76.089996,75.190002,74.290001,76.110001,77.279999,75.389999,73.169998,71.019997,71.459999,...,3.758,3.825,3.818,3.775,3.692,3.799,3.865,3.813,3.829,4.151
2022-12-23,77.489998,78.290001,76.089996,75.190002,74.290001,76.110001,77.279999,75.389999,73.169998,71.019997,...,3.706,3.758,3.825,3.818,3.775,3.692,3.799,3.865,3.813,3.829
2022-12-27,79.559998,77.489998,78.290001,76.089996,75.190002,74.290001,76.110001,77.279999,75.389999,73.169998,...,3.691,3.706,3.758,3.825,3.818,3.775,3.692,3.799,3.865,3.813


In [38]:
matrix_features_sector

,ATVI,CHTR,CL=F,CMCSA,DIS,DISH,EA,FOX,FOXA,GOOG,...,OMC,PARA,T,TMUS,TTWO,TWTR,VZ,WBD,^GSPC,^tnx
2018-01-22,71.300003,370.250000,63.490002,42.889999,111.099998,46.259998,117.989998,NaN,NaN,57.790501,...,76.360001,58.250000,28.602718,63.939999,118.400002,23.320000,53.459999,26.330000,2832.969971,2.665
2018-01-23,71.550003,369.250000,64.470001,42.439999,110.410004,46.330002,117.669998,NaN,NaN,58.498501,...,75.500000,57.520000,28.089125,64.529999,116.809998,22.750000,53.230000,25.700001,2839.129883,2.624
2018-01-24,70.080002,376.179993,65.610001,42.990002,110.500000,47.730000,114.660004,NaN,NaN,58.212002,...,76.580002,59.230000,27.960726,65.000000,116.239998,22.370001,54.220001,26.010000,2837.540039,2.654
2018-01-25,70.820000,375.140015,65.510002,42.139999,110.550003,47.029999,114.500000,NaN,NaN,58.518501,...,76.680000,59.270000,28.285498,64.389999,117.199997,22.160000,54.290001,25.730000,2839.250000,2.621
2018-01-26,72.519997,377.000000,66.139999,42.799999,112.190002,47.189999,115.199997,NaN,NaN,58.792000,...,77.650002,60.000000,28.564955,65.279999,119.370003,24.270000,54.720001,26.040001,2872.870117,2.662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-20,75.879997,309.290009,76.089996,34.410000,87.019997,13.820000,120.419998,28.100000,29.950001,89.629997,...,77.809998,16.879999,18.070000,140.339996,98.110001,NaN,36.990002,9.030000,3821.620117,3.684
2022-12-21,75.889999,321.339996,78.290001,35.080002,86.919998,13.920000,122.500000,28.389999,30.330000,90.250000,...,78.769997,16.879999,18.150000,141.470001,99.879997,NaN,37.779999,9.040000,3878.439941,3.684
2022-12-22,75.699997,323.880005,77.489998,34.820000,86.669998,14.140000,121.419998,28.540001,30.500000,88.260002,...,78.779999,16.940001,18.270000,139.529999,99.269997,NaN,38.310001,9.230000,3822.389893,3.669
2022-12-23,75.949997,333.920013,79.559998,35.139999,88.010002,14.460000,121.510002,28.850000,30.760000,89.809998,...,80.500000,16.660000,18.309999,140.020004,99.830002,NaN,38.410000,9.170000,3844.820068,3.751


In [124]:
Results

,Ticker,Date,Observed,Predicted for,Predicted,Recommended,Return %,Sector,Model
1,TTWO,2022-12-01,106.400002,2022-12-01,105.488589,Buy,0.671775,Communication Services,Lasso()
2,TTWO,2022-12-02,108.760002,2022-12-02,107.006165,Buy,2.218046,Communication Services,Lasso()
3,TTWO,2022-12-05,106.980003,2022-12-05,108.495991,Buy,-1.636630,Communication Services,Lasso()
4,TTWO,2022-12-06,101.269997,2022-12-06,106.770016,Sell,5.638399,Communication Services,Lasso()
5,TTWO,2022-12-07,99.489998,2022-12-07,101.419699,Sell,1.789123,Communication Services,Lasso()
...,...,...,...,...,...,...,...,...,...
17,TTWO,2022-12-23,99.830002,2022-12-23,99.783072,Sell,-0.560959,Communication Services,Lasso()
18,TTWO,2022-12-27,99.379997,2022-12-27,100.662096,Buy,-0.450771,Communication Services,Lasso()
19,TTWO,2022-12-28,99.690002,2022-12-28,100.489112,Sell,-0.310969,Communication Services,Lasso()
20,TTWO,2022-12-29,NaN,2022-12-29,99.646465,Sell,NaN,Communication Services,Lasso()


In [40]:
print(Results.dtypes)

Ticker                   object
Date             datetime64[ns]
Observed                float64
Predicted for    datetime64[ns]
Predicted               float64
Recommended              object
Return %                float64
Sector                   object
Model                    object
dtype: object


In [84]:
print(Results.iloc[-1])

Ticker                             TTWO
Date                2022-12-28 00:00:00
Observed                            NaN
Predicted for       2022-12-28 00:00:00
Predicted                    100.490806
Recommended                        Sell
Return %                            NaN
Sector           Communication Services
Model                           Lasso()
Name: 20, dtype: object


In [72]:
save_dataframe_to_bucket(Results, "Results_2.csv")
Results_2=read_dataframe_from_bucket("Results_2.csv")
Results_2['Date'] =  pd.to_datetime(Results_2['Date'], infer_datetime_format=True)
Results_2['Predicted for'] =  pd.to_datetime(Results_2['Predicted for'], infer_datetime_format=True)
print(Results_2.dtypes)

Ticker                   object
Date             datetime64[ns]
Observed                float64
Predicted for    datetime64[ns]
Predicted               float64
Recommended              object
Return %                float64
Sector                   object
Model                    object
dtype: object


In [73]:
Results_2

,Ticker,Date,Observed,Predicted for,Predicted,Recommended,Return %,Sector,Model
0,TTWO,2022-11-30,105.690002,2022-11-30,100.055661,Sell,-4.106352,Communication Services,Lasso()
1,TTWO,2022-12-01,106.400002,2022-12-01,105.485446,Buy,0.671775,Communication Services,Lasso()
2,TTWO,2022-12-02,108.760002,2022-12-02,106.993596,Buy,2.218046,Communication Services,Lasso()
3,TTWO,2022-12-05,106.980003,2022-12-05,108.497596,Buy,-1.636630,Communication Services,Lasso()
4,TTWO,2022-12-06,101.269997,2022-12-06,106.790140,Sell,5.638399,Communication Services,Lasso()
...,...,...,...,...,...,...,...,...,...
15,TTWO,2022-12-21,99.879997,2022-12-21,99.424772,Sell,-1.772123,Communication Services,Lasso()
16,TTWO,2022-12-22,99.269997,2022-12-22,99.797860,Buy,-0.610734,Communication Services,Lasso()
17,TTWO,2022-12-23,99.830002,2022-12-23,99.798034,Buy,0.564123,Communication Services,Lasso()
18,TTWO,2022-12-27,99.379997,2022-12-27,100.668609,Buy,-0.450771,Communication Services,Lasso()


In [37]:
#Testing Unique function
#there is multiple way to get unique items in array. np.unique is one of them
unique_SP500_tickers=np.unique(SP500_list[:,0])
unique_SP500_sectors=np.unique(SP500_list[:,1])
unique_SP500_industries=np.unique(SP500_list[:,2])
print(len(unique_SP500_tickers))
print(len(unique_SP500_sectors))
print(len(unique_SP500_industries))

503
11
114


In [38]:
#test for function initialize data - we want to check the test date last
print(f"days = {days}")
print(f"lags = {lags}")
print(f"nb_predict_days={nb_predict_days}")
yesterday, start_date, train_date_start, train_date_last, test_date_start, test_date_last,days=initialize_data(days,lags,nb_predict_days)


print(f"Model Input ->predict days {nb_predict_days}")
print(f"Model Period->yesterday {yesterday}")
print(f"Model period->train_date_start {train_date_start}")
print(f"Model Period->train_date_last {train_date_last}")
print(f"Model Period->test_date_start {test_date_start}")
print(f"Model Period->test_date_last {test_date_last}")



days = 1800
lags = 30
nb_predict_days=30
Model Input ->predict days 30
Model Period->yesterday 2022-12-28 20:13:06.946795
Model period->train_date_start 2018-01-23
Model Period->train_date_last 2022-11-27
Model Period->test_date_start 2022-11-28
Model Period->test_date_last 2022-12-28


In [39]:
#This is to test the filter function in numpy - here we're extracting all tickers for a sector
#ticker='TTWO'  #not required below
sector="Communication Services"
filename_json = "SP500.json"
SP500_list = read_list(filename_json)
SP500_list = np.array(SP500_list)  # we need an array


sub_list_tickers = np.array(SP500_list)
fltr = np.asarray([sector])
results = sub_list_tickers[np.in1d(sub_list_tickers[:, 1], fltr)]
print(results)
print(results[:, 0])
# on veut extraire les tickers donc toutes les rangees (0:0) et la colonne 0 donc (0:1)

print(results[:, 0].tolist())

[['ATVI' 'Communication Services' 'Electronic Gaming & Multimedia']
 ['CHTR' 'Communication Services' 'Telecom Services']
 ['CMCSA' 'Communication Services' 'Telecom Services']
 ['DIS' 'Communication Services' 'Entertainment']
 ['DISH' 'Communication Services' 'Telecom Services']
 ['EA' 'Communication Services' 'Electronic Gaming & Multimedia']
 ['FOX' 'Communication Services' 'Entertainment']
 ['FOXA' 'Communication Services' 'Entertainment']
 ['GOOG' 'Communication Services' 'Internet Content & Information']
 ['GOOGL' 'Communication Services' 'Internet Content & Information']
 ['IPG' 'Communication Services' 'Advertising Agencies']
 ['LUMN' 'Communication Services' 'Telecom Services']
 ['LYV' 'Communication Services' 'Entertainment']
 ['META' 'Communication Services' 'Internet Content & Information']
 ['MTCH' 'Communication Services' 'Internet Content & Information']
 ['NFLX' 'Communication Services' 'Entertainment']
 ['NWS' 'Communication Services' 'Entertainment']
 ['NWSA' 'Communi

In [40]:
#test si.get_data does not take last date therefore +timedelta(1)
print(start_date)
print(yesterday)
si.get_data('aapl', start_date, yesterday+timedelta(1))

2018-01-23 20:13:06.946795
2022-12-28 20:13:06.946795


,open,high,low,close,adjclose,volume,ticker
2018-01-23,44.325001,44.860001,44.205002,44.259998,42.086819,130756400,AAPL
2018-01-24,44.312500,44.325001,43.299999,43.555000,41.416439,204420400,AAPL
2018-01-25,43.627499,43.737499,42.632500,42.777500,40.677116,166116000,AAPL
2018-01-26,43.000000,43.000000,42.514999,42.877499,40.772202,156572000,AAPL
2018-01-29,42.540001,42.540001,41.767502,41.990002,39.928284,202561600,AAPL
...,...,...,...,...,...,...,...
2022-12-20,131.389999,133.250000,129.889999,132.300003,132.300003,77432800,AAPL
2022-12-21,132.979996,136.809998,132.750000,135.449997,135.449997,85928000,AAPL
2022-12-22,134.350006,134.559998,130.300003,132.229996,132.229996,77852100,AAPL
2022-12-23,130.919998,132.419998,129.639999,131.860001,131.860001,63771000,AAPL


In [41]:
#test the ffill NA function using tweeter data in december that was empty
test=si.get_data('twtr')
test=test.fillna(method='ffill')
test

,open,high,low,close,adjclose,volume,ticker
2013-11-07,45.099998,50.090000,44.000000,44.900002,44.900002,117701670.0,TWTR
2013-11-08,45.930000,46.939999,40.685001,41.650002,41.650002,27925307.0,TWTR
2013-11-11,40.500000,43.000000,39.400002,42.900002,42.900002,16113941.0,TWTR
2013-11-12,43.660000,43.779999,41.830002,41.900002,41.900002,6316755.0,TWTR
2013-11-13,41.029999,42.869999,40.759998,42.599998,42.599998,8688325.0,TWTR
...,...,...,...,...,...,...,...
2022-12-19,53.910000,54.000000,53.700001,53.700001,53.700001,136345128.0,TWTR
2022-12-20,53.910000,54.000000,53.700001,53.700001,53.700001,136345128.0,TWTR
2022-12-21,53.910000,54.000000,53.700001,53.700001,53.700001,136345128.0,TWTR
2022-12-22,53.910000,54.000000,53.700001,53.700001,53.700001,136345128.0,TWTR


In [42]:
pip install --upgrade google-cloud-logging

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.2/195.2 KB 3.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: google-cloud-logging
    Found existing installation: google-cloud-logging 3.3.1
    Uninstalling google-cloud-logging-3.3.1:
      Successfully uninstalled google-cloud-logging-3.3.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Imports the Google Cloud client library
from google.cloud import logging

# Instantiates a client
logging_client = logging.Client()

# The name of the log to write to
log_name = "my-log"
# Selects the log to write to
logger = logging_client.logger(log_name)



logger.log_text(f"Model Sector -> {sector}")
logger.log_text(f"Model Input ->years {years}")
logger.log_text(f"Model Input ->ticker list {ticker_list_for_models}")
logger.log_text(f"Model Input ->ticker list extended {tickers_in_sector_extended}")
logger.log_text(f"Model Input->lags {lags}")
logger.log_text(f"Model Input ->predict days {nb_predict_days}")
logger.log_text(f"Model Period->yesterday {yesterday}")
logger.log_text(f"Model period->train_date_start {train_date_start}")
logger.log_text(f"Model Period->train_date_last {train_date_last}")
logger.log_text(f"Model Period->test_date_start {test_date_start}")
logger.log_text(f"Model Period->test_date_last {test_date_last}")

logger.log_text(f"Model This is the tickers for our model {ticker_list_for_models}")
logger.log_text(f"Model This is the additional data  we add to the tickers for the model {additional_data}")




In [96]:
#code given by chatGPT
import yfinance as yf

# Get the S&P 500 index
sp500 = yf.SP500()

# Get the ticker symbols for the S&P 500 index
all_tickers = sp500.tickers
print(all_tickers)

AttributeError: module 'yfinance' has no attribute 'SP500'

TypeError: 'module' object is not callable

In [101]:
SP500

<module 'yfinance.tickers' from '/home/steve/.local/lib/python3.10/site-packages/yfinance/tickers.py'>

In [4]:

model_file_name="SP500.json"
upload_file_to_bucket(model_file_name)